# exploring allele1/2 data interactively

carefully defining compartments, expected, and looking at the allele1/2 interaction profile

### make notebook feel like sublime
stollen from the great and mighty internets
multiline editing and some nice key-bindings

In [2]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython) {
    
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
    
        var cells = IPython.notebook.get_cells();
    
    
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# i had to install this beauty https://github.com/matplotlib/ipympl
# to make following to work ...
%matplotlib widget
import ipywidgets as widgets

# %matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import os
import re
import numpy as np
import pandas as pd
import cooler

import itertools

In [5]:
import seaborn as sns

In [6]:
import bioframe
from bioframe.io import resources

In [7]:
# # make pandas display entire dataframes
# pd.set_option("display.max_rows", None, "display.max_columns", None)

In [8]:
# cooltools used dekkerlab-fork, develop-branch, commit-af508bba9c6577b8271b716ba27c330bead15981

from cooler.tools import split, partition
from functools import partial
from cooltools import expected

In [9]:
from cooltools import numutils
from cooltools.eigdecomp import cooler_cis_eig

In [10]:
from helper_module import fillcolor_compartment_style, to_uscs

In [11]:
cooler_names = [#'all-allele-spe.galGal5.mapq_30.1000.mcool',
#'all-allele-spe-high-cis.galGal5.mapq_30.1000.mcool',
    'all-allele-spe-2percent-highcis.galGal5.mapq_30.1000.mcool',
    'all-allele-spe-1percent-highcis.galGal5.mapq_30.1000.mcool',
'MNP-DT40-1-3-16-2p-R1-T1__galGal5.galGal5.mapq_30.1000.mcool',
'MNP-DT40-1-3-16-R1-T1__galGal5.galGal5.mapq_30.1000.mcool',
'MNP-DT40-1-3-16-R2-T1__galGal5.galGal5.mapq_30.1000.mcool',
'MNP-DT40-1-3-17-2p-R1-T1__galGal5.galGal5.mapq_30.1000.mcool',
'MNP-DT40-1-3-17-R1-T1__galGal5.galGal5.mapq_30.1000.mcool',
'MNP-DT40-1-3-3-R1-T1__galGal5.galGal5.mapq_30.1000.mcool',
'MNP-DT40-WT1-R1-T1__galGal5.galGal5.mapq_30.1000.mcool',
'MNP-DT40-WT2-R1-T1__galGal5.galGal5.mapq_30.1000.mcool']

### generate some nicer sample names from coolers ...

In [12]:
# multiqc learnings here, btw!...
renamings = {"all-allele-spe":"pool",
            ".galGal5.mapq_30.1000.mcool":"",
            "MNP-DT40-":"",
            "-T1__galGal5":"",
            "-R1":"",
            "1-3-":"clone-"}

sample_names = []
for c in cooler_names:
    for rf,rt in renamings.items():
        c = re.sub(rf,rt,c)
    sample_names.append(c)

# sample_names

# form a df of samples here:
samples = pd.DataFrame({"sample":sample_names,"mcool":cooler_names})

samples

,sample,mcool
0,pool-2percent-highcis,all-allele-spe-2percent-highcis.galGal5.mapq_3...
1,pool-1percent-highcis,all-allele-spe-1percent-highcis.galGal5.mapq_3...
2,clone-16-2p,MNP-DT40-1-3-16-2p-R1-T1__galGal5.galGal5.mapq...
3,clone-16,MNP-DT40-1-3-16-R1-T1__galGal5.galGal5.mapq_30...
4,clone-16-R2,MNP-DT40-1-3-16-R2-T1__galGal5.galGal5.mapq_30...
5,clone-17-2p,MNP-DT40-1-3-17-2p-R1-T1__galGal5.galGal5.mapq...
6,clone-17,MNP-DT40-1-3-17-R1-T1__galGal5.galGal5.mapq_30...
7,clone-3,MNP-DT40-1-3-3-R1-T1__galGal5.galGal5.mapq_30....
8,WT1,MNP-DT40-WT1-R1-T1__galGal5.galGal5.mapq_30.10...
9,WT2,MNP-DT40-WT2-R1-T1__galGal5.galGal5.mapq_30.10...


In [13]:
def bar_signal(oe,signal,thresh=0.1):
    oe1 = np.nanmean(oe[signal >= thresh])
    oe2 = np.nanmean(oe[(signal >= -thresh)&(signal < thresh)])
    oe3 = np.nanmean(oe[signal < -thresh])
    return [oe1,oe2,oe3]

#     #     [a1A,a2A,a1B,a2B]
#     # ...
#     a1A = np.nanmean(oe1_vis[cmpp >= 0.1])
#     a1AB = np.nanmean(oe1_vis[(cmpp >= -0.1)&(cmpp < 0.1)])
#     a1B = np.nanmean(oe1_vis[cmpp < -0.1])

#     a2A = np.nanmean(oe2_vis[cmpp >= 0.1])
#     a2AB = np.nanmean(oe2_vis[(cmpp >= -0.1)&(cmpp < 0.1)])
#     a2B = np.nanmean(oe2_vis[cmpp < -0.1])

#     axbar.bar(range(6),[a1A,a1AB,a1B,a2A,a2AB,a2B],color=list('rrrggg'))


In [14]:
# READ SOME COMPARTMENTS IN ...
# we have only 100kb externally called compartments ...
binsize = 100_000
res_human = f"{int(binsize/1000)}kb"

comp_path = "/home/venevs/DOTS_TESTING/alv-chicken/MNP/analysis/compartment"
comp_name = f"poolMNP-bettercis.{res_human}.eigs.cis.vecs.txt"

cmp_vals50kb = None

cmp = pd.read_csv(os.path.join(comp_path,comp_name),sep='\t')

chrom="chr1"
# get chromosomal compartments and interpolated version of those ...
cmp_vals = cmp[cmp["chrom"]==chrom]['E1'].values
# # interpolated_cmp = get_proper_interpolated_compartments(clr, cmp_vals, chrom)
# cmp_vals = cmp_vals[::10]

In [38]:
# READ SOME COOLERS ...
binsize = 1_000_000
res_human = f"{int(binsize/1000)}kb"

# refres compartments derived earlier ...
# get chromosomal compartments and interpolated version of those ...
cmp_vals = cmp[cmp["chrom"]==chrom]['E1'].values


# oldref-20200617
# cool_path = "/home/venevs/DOTS_TESTING/alv-chicken/oldref-20200410"
cool_path = "/home/venevs/DOTS_TESTING/alv-chicken/oldref-20200617"
get_cpath = lambda c: os.path.join(cool_path,c+f"::/resolutions/{binsize}")

samples["cool"] = [cooler.Cooler(get_cpath(c)) for c in samples["mcool"]]


samples["sum"] = [s.info["sum"] for s in samples["cool"]]

In [39]:
# make a bunch of decisions/assignments, based on the binsize of the cooler:

if binsize == 50_000:
    cmp_vals_tmp = np.zeros(2*len(cmp_vals))
    cmp_vals_tmp[0::2] = cmp_vals
    cmp_vals_tmp[1::2] = cmp_vals
    cmp_vals = cmp_vals_tmp
    bad_bins =  np.r_[np.arange(180,189),np.arange(650,655),np.arange(1440,1446)]
elif binsize == 100_000:
    bad_bins =  np.r_[np.arange(90,95),np.arange(325,330),np.arange(720,723)]
    cmp_vals = cmp_vals
elif binsize == 250_000:
    bad_bins =  np.array([9*4,9*4+1,32*4,32*4+1,72*4,72*4+1])
    if cmp_vals50kb is None:
        cmp_vals = cmp_vals[::3] # probably wrong ...
    else:
        cmp_vals = cmp_vals50kb[::5].copy()
elif binsize == 500_000:
    bad_bins =  np.array([9*2,9*2+1,32*2,32*2+1,72*2,72*2+1])
    cmp_vals = cmp_vals[::5]
elif binsize == 1_000_000:
    bad_bins =  np.array([9,32,72])
    cmp_vals = cmp_vals[::10]
else:
    print("uh oh - dunno !")
#  
# insertion site, bin-id:    
#

if binsize == 50_000:
    ins_row = 1451 # obvious only in raw data, balaned has al1 and al2 separated !
elif binsize == 100_000:
    ins_row = 725 # 290*2.5
elif binsize == 250_000:
    ins_row = 290 # ok
elif binsize == 500_000:
    ins_row = 145
elif binsize == 1_000_000:
    ins_row = 72
else:
    print("uh oh - dunno where is the insertion!")

In [40]:
clr, = samples["cool"][samples["sample"] == "pool-2percent-highcis"].values

parm = ("chr1",0,74626886)
qarm = ("chr1",75126886,196202544)
centro = ("chr1",74626886,75126886)
balance = True

# bad_bins = None
ignore_diags = 2
theregion=parm
theregion_name="parm"

bad_bin_region = ("chr1",9000000,9500000)

weight_name = "weight"
weight1 = weight_name + "1"
weight2 = weight_name + "2"
transforms = {"balanced": lambda p: p["count"] * p[weight1] * p[weight2]}

### load replication timing in here !

In [41]:
from bioframe import io

timfname = "/home/venevs/DOTS_TESTING/alv-chicken/timing_wt.bed"

bins = clr.bins().fetch(chrom).copy()

val_rt = io.read_table(timfname,
                       schema=bioframe.schemas.BEDGRAPH_FIELDS,
                       skiprows=1)
val_rt = val_rt.rename({0:"chrom",1:"start",2:"end",3:"rt"},
                       axis=1)

# fnd = plt.figure(figsize=(10,2))
# val_rt.plot(x=1,y=3,ax=plt.gca(),lw=1)

# overlap RT signal with our bins ...
binned_rt = bioframe.ops.overlap(bins, val_rt, suffixes=['', '_rt'])
# freaking pd.NA !? can't cast them to float ... why ? why are they there in the first place ?!
binned_rt["value_rt"] = binned_rt["value_rt"].apply(lambda x: np.nan if x is pd.NA else x)

binned_rt = binned_rt \
                .drop(labels=["chrom_rt","start_rt","end_rt"],axis=1) \
                .astype({"start":int,"end":int,"weight":float})

# finally :
binned_rt = binned_rt.groupby(["chrom","start","end"],observed=True,sort=False).mean()

### compute compartments per arm masking bad bins ...

In [42]:
# from cooltools import eigdecomp
# pcmp = cmp_vals[slice(*clr.extent(parm))]
# qcmp = cmp_vals[slice(*clr.extent(qarm))]
# phasing_trck = np.concatenate((pcmp,qcmp))

# insert existing eigen track as a phasing track:
bins = clr.bins()[:].copy()
# no support for bad bins in eigdecomp ...
cmp_phase = np.zeros(len(bins))
cmp_phase[:len(cmp_vals)] = cmp_vals
bins.insert(4,"cmp",cmp_phase)

eigvals1, eigvec_table1 = cooler_cis_eig(
    clr=clr,
    bins=bins,
    regions= [parm, qarm],
    n_eigs=2,
    bad_bins = bad_bins,
    phasing_track_col="cmp",
    clip_percentile=99.9,
    sort_metric=None,
)

# plot it, just in case:
f = plt.figure(figsize=(10,3))
ax = f.add_subplot(2,1,1)
axrt = f.add_subplot(2,1,2)
print(len(eigvec_table1))
fillcolor_compartment_style(eigvec_table1["E1"], ax)
# fillcolor_compartment_style(cmp_vals, ax)
for bb in bad_bins:
    ax.axvline(bb)

fillcolor_compartment_style(binned_rt["value_rt"], axrt)

    
# redefine cmp_vals:
cmp_vals = eigvec_table1[eigvec_table1["chrom"]==chrom]["E1"].values

if binsize == 50_000:
    cmp_vals50kb = cmp_vals.copy()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1040


### correlate RT and EV1 to check if apparent shift is real

In [43]:
ftftrf = plt.figure(figsize=(3,3))

x = eigvec_table1["E1"].iloc[slice(*clr.extent(chrom))].values
y = binned_rt["value_rt"].values

ax = plt.gca()

@widgets.interact(shhh=(0,30))
def update(shhh=0):
    ax.clear()
    
    if shhh == 0:
        xv = x[:].copy()
        yv = y[:].copy()
    else:
        xv = x[shhh:].copy()
        yv = y[:-shhh].copy()
    rrr = pd.DataFrame({"x":xv,"y":yv}).corr().loc["x","y"]
    #     ax.hexbin(xv,yv,alpha=0.3,label=f"{rrr}")
    ax.hexbin(xv,yv,gridsize=(30,18) ,cmap="YlOrBr")
    ax.set_title(f"{rrr}")
    #     plt.legend()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='shhh', max=30), Output()), _dom_classes=('widget-interac…

In [44]:
# plot it, just in case:
fаа = plt.figure(figsize=(4,4))
# ax = f.add_subplot()
# print(len(eigvec_table1))
# fillcolor_compartment_style(eigvec_table1["E1"], ax)
# fillcolor_compartment_style(cmp_vals, ax)
# for bb in bad_bins:
#     ax.axvline(bb)

cmap = matplotlib.cm.YlOrBr
cmap.set_bad('grey',.5)
mat = clr.matrix(balance=False).fetch(parm)
img = plt.imshow(np.log(mat),cmap="YlOrBr",interpolation='nearest')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### calculate expecteds the new way ...

In [45]:
import multiprocess as mp

nproc = 4

# matching regions: p-arm        pq-inter arm      q-arm            chr1-wide ...
left_regions  = [(*parm,'parm'),(*parm,'parm'),(*qarm,'qarm'),("chr1",0,clr.chromsizes["chr1"],"chr1")]
right_regions = [(*parm,'parm'),(*qarm,'qarm'),(*qarm,'qarm'),("chr1",0,clr.chromsizes["chr1"],"chr1")]
# execution details
if nproc > 1:
    pool = mp.Pool(nproc)
    map_ = pool.map
else:
    map_ = map

exp_df_bad = expected.diagsum_asymm(clr,
                 regions1 = left_regions,
                 regions2 = right_regions,
                 transforms=transforms,
                 weight_name='weight',
                 bad_bins=bad_bins,
                 map = map_)
exp_df_bad["balanced.avg"] = exp_df_bad["balanced.sum"]/exp_df_bad["n_valid"]
exp_grp_bad = exp_df_bad.groupby(["region1","region2"])
# extract several expecteds for different regions ...
parm_exp_bad = exp_grp_bad.get_group(("parm","parm"))
pqarm_exp_bad = exp_grp_bad.get_group(("parm","qarm"))
# qarm_exp_bad = exp_grp_bad.get_group(("qarm","qarm"))
chr1_exp_bad = exp_grp_bad.get_group(("chr1","chr1"))


exp_df_bad_raw = expected.diagsum_asymm(clr,
                     regions1 = left_regions,
                     regions2 = right_regions,
                     transforms={},
                     weight_name=None,
                     bad_bins=bad_bins,
                     map = map_)
exp_df_bad_raw["count.avg"] = exp_df_bad_raw["count.sum"]/exp_df_bad_raw["n_valid"]
exp_df_bad_raw_grp = exp_df_bad_raw.groupby(["region1","region2"])
# extract several expecteds for different regions ...
parm_exp_bad_raw = exp_df_bad_raw_grp.get_group(("parm","parm"))
pqarm_exp_bad_raw = exp_df_bad_raw_grp.get_group(("parm","qarm"))
# qarm_exp_bad_raw = exp_df_bad_raw_grp.get_group(("qarm","qarm"))
chr1_exp_bad_raw = exp_df_bad_raw_grp.get_group(("chr1","chr1"))


pool.close()


## now to allele1/2 ...

all about insertion site and such ...

In [46]:
# shape of the inter-arm heatmap ...
lo,hi = clr.extent(parm)
shape0 = hi-lo
lo,hi = clr.extent(qarm)
shape1 = hi-lo
shape = (shape0,shape1)
print(shape)

chr1_parm = parm

psize = shape0
qsize = shape1

get_cr = lambda e, shape : (e[shape[0]-1::-1], e[shape[0]-1:])

# balanced expecteds ...
pqexp = numutils.LazyToeplitz(*get_cr(pqarm_exp_bad["balanced.avg"].values, shape))
pexp = numutils.LazyToeplitz(parm_exp_bad["balanced.avg"])
# qexp = numutils.LazyToeplitz(qarm_exp_bad["balanced.avg"])
chr1exp = numutils.LazyToeplitz(chr1_exp_bad["balanced.avg"])

# raw expecteds ...
pqexp_raw = numutils.LazyToeplitz(*get_cr(pqarm_exp_bad_raw["count.avg"].values, shape))
pexp_raw = numutils.LazyToeplitz(parm_exp_bad_raw["count.avg"])
# qexp_raw = numutils.LazyToeplitz(qarm_exp_bad_raw["count.avg"])
chr1exp_raw = numutils.LazyToeplitz(chr1_exp_bad_raw["count.avg"])


chr1_parm = to_uscs(parm)
chr1_qarm = to_uscs(qarm)

# some very stupid check:
# actually this check breaks at higher resolutions - i.e. small binsize
#  because there is a gap between p and q arms - which becomes > 1 bin eventually
if (shape0 + shape1) == chr1exp.shape[0]:
    print(
        np.allclose(np.r_[chr1exp[:shape0,ins_row], chr1exp[-shape1:,ins_row]] , chr1exp[:,ins_row])
    )
else:
    print(f"make sure at a given binsize {binsize}, centromere is > 1 bin ...")


# the following must be always true, though - as it accounts for the centromere size
assumption = np.diff(clr.extent(parm))+np.diff(clr.extent(centro))+np.diff(clr.extent(qarm))-2 == np.diff(clr.extent(chrom))
assert assumption

(75, 122)
True


### let's extract allele1/2-related observed, expected and weights ...

In [47]:
  
# WE ARE NOT GOING TO USE THE WEIGHTS FOR THE ALLELE1/2 ITSELF ...
# AS IT IS UNCLEAR - WHAT THEY REFLECT - ALLELE1 AND ALLELE2 AREN'T COMPARABLE
# THEMSELVES , ONLY HOW THEY INTERACT WITH THE REST OF THE GENOME ...
# wa1, = clr.bins().fetch("allele1")["weight"]
# wa2, = clr.bins().fetch("allele2")["weight"]
                

# allele1/2 observed - raw:
obs_r_a1_1p, = clr.matrix(balance=False).fetch("allele1",chr1_parm)
obs_r_a1_1q, = clr.matrix(balance=False).fetch("allele1",chr1_qarm)

obs_r_a2_1p, = clr.matrix(balance=False).fetch("allele2",chr1_parm)
obs_r_a2_1q, = clr.matrix(balance=False).fetch("allele2",chr1_qarm)

# balancing weights p-arm with bad_bins applied ...
wp = clr.bins().fetch(chr1_parm)[weight_name].values
lo,hi = clr.extent(chr1_parm)
bad_bins_parm = bad_bins[(bad_bins>=lo)&(bad_bins<hi)] - lo
if len(bad_bins_parm) > 0:
    wp[bad_bins_parm] = np.nan

# balancing weights q-arm with bad_bins masking applied ...
wq = clr.bins().fetch(chr1_qarm)[weight_name].values
lo,hi = clr.extent(chr1_parm)
bad_bins_parm = bad_bins[(bad_bins>=lo)&(bad_bins<hi)] - lo
if len(bad_bins_parm) > 0:
    wp[bad_bins_parm] = np.nan


# # allele1/2 observed - balanced:
# obs_b_a1_1p = wp * obs_r_a1_1p
# obs_b_a1_1q = wp * obs_r_a1_1q

# obs_b_a2_1p = wq * obs_r_a2_1p
# obs_b_a2_1q = wq * obs_r_a2_1q

# p-arm expected balanced al1 and al2 are the same because insertion sites is really close (+/- 1kb):
exp_b_a1a2_1p = pexp[:,ins_row].flatten()
exp_b_a1a2_1p_chr1 = chr1exp[:psize,ins_row].flatten()
# p-arm expected raw al1 and al2 are the same because insertion sites is really close (+/- 1kb):
exp_r_a1a2_1p = pexp_raw[:,ins_row].flatten()
exp_r_a1a2_1p_chr1 = chr1exp_raw[:psize,ins_row].flatten()

# q-arm expected balanced al1 and al2 are the same because insertion sites is really close (+/- 1kb):
exp_b_a1a2_1q = pqexp[ins_row,:].flatten()
exp_b_a1a2_1q_chr1 = chr1exp[-qsize:,ins_row].flatten()
# q-arm expected balanced al1 and al2 are the same because insertion sites is really close (+/- 1kb):
exp_r_a1a2_1q = pqexp_raw[ins_row,:].flatten()
exp_r_a1a2_1q_chr1 = chr1exp_raw[-qsize:,ins_row].flatten()

# if balanced:
#     wa1, = clr.bins().fetch("allele1")["weight"]
#     wa2, = clr.bins().fetch("allele2")["weight"]
#     # print(wa1,wa2)725


In [48]:

# visualize compartments:
f = plt.figure(figsize=(9,5),constrained_layout=True)
sp = gridspec.GridSpec(ncols=4,
                       nrows=3,
                       height_ratios=[2,1,1],
                       width_ratios=[0.3*psize,psize,qsize,0.3*psize],
                       figure=f
                      )

axp = f.add_subplot(sp[0,1])
axq = f.add_subplot(sp[0,2],sharey=axp)

axpc = f.add_subplot(sp[1,1],sharex=axp)
axqc = f.add_subplot(sp[1,2],sharex=axq,sharey=axpc)

axprt = f.add_subplot(sp[2,1],sharex=axp)
axqrt = f.add_subplot(sp[2,2],sharex=axq,sharey=axprt)


axpcb = f.add_subplot(sp[1,0])
axqcb = f.add_subplot(sp[1,3])

axprtb = f.add_subplot(sp[2,0])
axqrtb = f.add_subplot(sp[2,3])


axp.tick_params(bottom=False,labelbottom=False)
axq.tick_params(bottom=False,labelbottom=False,left=False,labelleft=False)
axpc.tick_params(bottom=False,labelbottom=False)
axqc.tick_params(bottom=False,labelbottom=False,left=False,labelleft=False)
axqrt.tick_params(left=False,labelleft=False)


clr = samples["cool"][0]

rtdf = binned_rt["value_rt"].reset_index(drop=True).values

p_range_zoom=widgets.IntRangeSlider(
        value=clr.extent(chr1_parm),
        min=0,
        max=clr.extent(chr1_parm)[1],
        step=1,
        continuous_update=False)

q_range_zoom=widgets.IntRangeSlider(
        value=clr.extent(chr1_qarm),
        min=clr.extent(chr1_qarm)[0],
        max=clr.extent(chr1_qarm)[1],
        step=1,
        continuous_update=False)

sub_frac = widgets.FloatSlider(
        value=0.65,
        min=0,
        max=1.0,
        step=0.05)

@widgets.interact(
        p_range_zoom = p_range_zoom,
        q_range_zoom = q_range_zoom,
        log=True,
        balanced=True,
        OE=False,
        samplez = samples["sample"].values,
        sub_frac = sub_frac
)
def update(p_range_zoom,
           q_range_zoom,
           log,
           balanced,
           OE,
           samplez,
           sub_frac
          ):
    # we are switching samples inside interactively ...
    clr, = samples["cool"][samples["sample"] == samplez].values

    axp.clear()
    axq.clear()
    
    axpcb.clear()
    axqcb.clear()
    axprtb.clear()
    axqrtb.clear()
    
    axpc.clear()
    axqc.clear()
    axprt.clear()
    axqrt.clear()
    # extract balancing weights for
    # alleles 1 and 2 - they are meaningless I guess
    # they only reflect - the total # of interactions
    # between allele1/2 and chr1

    # p-arm data:
    d1p = np.random.binomial(obs_r_a1_1p,sub_frac).T
    d2p = obs_r_a2_1p
    if balanced:
        d1p = d1p * wp
        d2p = d2p * wp
        ep12 = exp_b_a1a2_1p
        epc12 = exp_b_a1a2_1p_chr1
    else:
        ep12 = exp_r_a1a2_1p
        epc12 = exp_r_a1a2_1p_chr1
    if OE:
        # only arm level expected ...
        d1p = d1p / ep12
        d2p = d2p / ep12
    # adjust everything to the visible region:
    xp = range(*p_range_zoom)
    # compartments:
    cmpp = cmp_vals[slice(*p_range_zoom)]
    rtpp = rtdf[slice(*p_range_zoom)]

    # q-arm data:
    d1q = np.random.binomial(obs_r_a1_1q,sub_frac).T
    d2q = obs_r_a2_1q
    if balanced:
        d1q = d1q * wq
        d2q = d2q * wq
        epq12 = exp_b_a1a2_1q
        epqc12 = exp_b_a1a2_1q_chr1
    else:
        epq12 = exp_r_a1a2_1q
        epqc12 = exp_r_a1a2_1q_chr1
    if OE:
        # only arm level expected ...
        d1q = d1q / epq12
        d2q = d2q / epq12
    # adjust everything to the visible region:
    xq = range(*q_range_zoom)
    cmpq = cmp_vals[slice(*q_range_zoom)]
    rtpq = rtdf[slice(*q_range_zoom)]

    axp.plot(xp, d1p[slice(*p_range_zoom)], 'gx', label="allele1", alpha=0.9)
    axp.plot(xp, d2p[slice(*p_range_zoom)], 'mx', label="allele2", alpha=0.9)
    if not OE:
        axp.plot(xp, ep12[slice(*p_range_zoom)], 'g-', alpha=0.5)
        axp.plot(xp, epc12[slice(*p_range_zoom)], 'r-', alpha=0.5)
    axp.set_xlim(*p_range_zoom)

    q_range_zoom_rel = [_-clr.offset(chr1_qarm) for _ in q_range_zoom]
    axq.plot(xq, d1q[slice(*q_range_zoom_rel)], 'gx', label="allele1", alpha=0.9)
    axq.plot(xq, d2q[slice(*q_range_zoom_rel)], 'mx', label="allele2", alpha=0.9)
    if not OE:
        axq.plot(xq, epq12[slice(*q_range_zoom_rel)], 'g-', alpha=0.5)
        axq.plot(xq, epqc12[slice(*q_range_zoom_rel)], 'r-', alpha=0.5)
    axq.set_xlim(*q_range_zoom)
    axq.legend(loc="best")

    if log:
        axp.set_yscale("log")
        axq.set_yscale("log")
    
    fillcolor_compartment_style(cmp_vals, axpc)
    fillcolor_compartment_style(cmp_vals, axqc)

    fillcolor_compartment_style(rtdf, axprt)
    fillcolor_compartment_style(rtdf, axqrt)
    
    axpcb.bar(range(6),bar_signal(d1p[[slice(*p_range_zoom)]],cmpp)+bar_signal(d2p[[slice(*p_range_zoom)]],cmpp),color=list('rkbrkb'))
    axqcb.bar(range(6),bar_signal(d1q[slice(*q_range_zoom_rel)],cmpq)+bar_signal(d2q[slice(*q_range_zoom_rel)],cmpq),color=list('rkbrkb'))

    axprtb.bar(range(6),bar_signal(d1p[slice(*p_range_zoom)],rtpp)+bar_signal(d2p[slice(*p_range_zoom)],rtpp),color=list('rkbrkb'))
    axqrtb.bar(range(6),bar_signal(d1q[slice(*q_range_zoom_rel)],rtpq)+bar_signal(d2q[slice(*q_range_zoom_rel)],rtpq),color=list('rkbrkb'))




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntRangeSlider(value=(0, 75), continuous_update=False, description='p_range_zoom', max=7…

In [49]:
# bar_signal??

# def bar_signal(oe,signal):
#     oe1 = np.nanmean(oe[signal >= 0.1])
#     oe2 = np.nanmean(oe[(signal >= -0.1)&(signal < 0.1)])
#     oe3 = np.nanmean(oe[signal < -0.1])
#     return [oe1,oe2,oe3]

In [50]:
## # test (1)

# difflist_a1 = []
# difflist_a2 = []
# At = []
# Bt = []
# num_iters = 10000

# for iteration in range(num_iters):
#     aa = np.random.permutation( oe_eigen_df_filtered['compartment'].values )
#     oe_eigen_df_filtered['randomized_comp'] = pd.Series(aa)
#     grp_by_rnd_cmp = oe_eigen_df_filtered.groupby('randomized_comp')
#     gt = grp_by_rnd_cmp['allele1','allele2'].mean()
#     # first test expected:
#     all1_test = gt['allele1']
#     all2_test = gt['allele2']
#     all1_ABdiff = all1_test['A'] - all1_test['B']
#     all2_ABdiff = all2_test['B'] - all2_test['A']
#     difflist_a1.append(all1_ABdiff)
#     difflist_a2.append(all2_ABdiff)
#     # second test expected:
#     cmpA_test = gt.loc['A']
#     cmpB_test = gt.loc['B']
#     Atest = cmpA_test['allele1'] - cmpA_test['allele2']
#     Btest = cmpB_test['allele2'] - cmpB_test['allele1']
#     At.append(Atest)
#     Bt.append(Btest)

# # test (2)
# # this would be a different test probably ...
# # start = 1
# # maximum = num_iters # like 10000 or so
# # diffAlist = []
# # while start <= maximum:
# #     aa = oe_eigen_df_filtered['compartment'].reindex(np.random.permutation(oe_eigen_df_filtered['compartment'].index)).reset_index(drop=True)
# #     oe_eigen_df_filtered['randomized_comp'] = pd.Series(aa)
# #     randa1 = oe_eigen_df_filtered.loc[oe_eigen_df_filtered['randomized_comp'] == 'A', 'allele1'].mean()
# #     randa2 = oe_eigen_df_filtered.loc[oe_eigen_df_filtered['randomized_comp'] == 'A', 'allele2'].mean()
# #     diffA = randa1 - randa2
# #     #print (diffA)
# #     diffAlist.append(diffA)
# #     start += 1



# visualize compartments:
fx = plt.figure(figsize=(4,2),constrained_layout=True)
spx = gridspec.GridSpec(ncols=2,
                       nrows=1,
                       width_ratios=[1,1],
                       figure=fx
                      )

axp = fx.add_subplot(spx[0])
axq = fx.add_subplot(spx[1])#,sharey=axp)


# axp.tick_params(bottom=False,labelbottom=False)
# axq.tick_params(bottom=False,labelbottom=False,left=False,labelleft=False)

clr = samples["cool"][0]

p_range_zoom=widgets.IntRangeSlider(
        value=clr.extent(chr1_parm),
        min=0,
        max=clr.extent(chr1_parm)[1],
        step=1,
        continuous_update=False)

q_range_zoom=widgets.IntRangeSlider(
        value=clr.extent(chr1_qarm),
        min=clr.extent(chr1_qarm)[0],
        max=clr.extent(chr1_qarm)[1],
        step=1,
        continuous_update=False)

sub_frac = widgets.FloatSlider(
        value=0.65,
        min=0,
        max=1.0,
        step=0.05)

threshold = widgets.FloatSlider(
        value=0.1,
        min=0,
        max=0.5,
        step=0.05)

@widgets.interact(
        p_range_zoom = p_range_zoom,
        q_range_zoom = q_range_zoom,
        balanced=True,
        OE=True,
        samplez = samples["sample"].values,
        sub_frac = sub_frac,
        threshold = threshold,
)
def update(p_range_zoom,
           q_range_zoom,
           balanced,
           OE,
           samplez,
           sub_frac,
           threshold,
          ):
    # we are switching samples inside interactively ...
    clr, = samples["cool"][samples["sample"] == samplez].values

    axp.clear()
    axq.clear()
    # extract balancing weights for
    # alleles 1 and 2 - they are meaningless I guess
    # they only reflect - the total # of interactions
    # between allele1/2 and chr1
    
    aa_list_p = []
    rnd_list_p = []
    aa_list_q = []
    rnd_list_q = []
    for ii in range(1):
        # p-arm data:
        d1p = np.random.binomial(obs_r_a1_1p,sub_frac).T
        d2p = obs_r_a2_1p
        if balanced:
            d1p = d1p * wp
            d2p = d2p * wp
            ep12 = exp_b_a1a2_1p
            epc12 = exp_b_a1a2_1p_chr1
        else:
            ep12 = exp_r_a1a2_1p
            epc12 = exp_r_a1a2_1p_chr1
        if OE:
            # only arm level expected ...
            d1p = d1p / ep12
            d2p = d2p / ep12
        # q-arm data:
        d1q = np.random.binomial(obs_r_a1_1q,sub_frac).T
        d2q = obs_r_a2_1q
        if balanced:
            d1q = d1q * wq
            d2q = d2q * wq
            epq12 = exp_b_a1a2_1q
            epqc12 = exp_b_a1a2_1q_chr1
        else:
            epq12 = exp_r_a1a2_1q
            epqc12 = exp_r_a1a2_1q_chr1
        if OE:
            # only arm level expected ...
            d1q = d1q / epq12
            d2q = d2q / epq12

        # adjust everything to the visible region:
        xp = range(*p_range_zoom)
        # compartments:
        cmpp = cmp_vals[slice(*p_range_zoom)]

        # adjust everything to the visible region:
        xq = range(*q_range_zoom)
        cmpq = cmp_vals[slice(*q_range_zoom)]

        aa1 = pd.DataFrame({"set": ["A1","AB1","B1"], "OE": bar_signal(d1p[slice(*p_range_zoom)],cmpp,threshold)})
        aa2 = pd.DataFrame({"set": ["A2","AB2","B2"], "OE": bar_signal(d2p[slice(*p_range_zoom)],cmpp,threshold)})
        aa_list_p.append(aa1)
        aa_list_p.append(aa2)
        rnd1 = random_test(d1p[slice(*p_range_zoom)],cmpp,iters=1000,thresh=threshold,cyclic=False)
        rnd1["set"] = rnd1["set"] + "1"
        rnd2 = random_test(d2p[slice(*p_range_zoom)],cmpp,iters=1000,thresh=threshold,cyclic=False)
        rnd2["set"] = rnd1["set"] + "2"
        rnd_list_p.append(rnd1)
        rnd_list_p.append(rnd2)

        q_range_zoom_rel = [_-clr.offset(chr1_qarm) for _ in q_range_zoom]
        aa1 = pd.DataFrame({"set": ["A1","AB1","B1"], "OE": bar_signal(d1q[slice(*q_range_zoom_rel)],cmpq,threshold)})
        aa2 = pd.DataFrame({"set": ["A2","AB2","B2"], "OE": bar_signal(d2q[slice(*q_range_zoom_rel)],cmpq,threshold)})
        aa_list_q.append(aa1)
        aa_list_q.append(aa2)
        rnd1 = random_test(d1q[slice(*q_range_zoom_rel)],cmpq,iters=1000,thresh=threshold,cyclic=False)
        rnd1["set"] = rnd1["set"] + "1"
        rnd2 = random_test(d2q[slice(*q_range_zoom_rel)],cmpq,iters=1000,thresh=threshold,cyclic=False)
        rnd2["set"] = rnd1["set"] + "2"
        rnd_list_q.append(rnd1)
        rnd_list_q.append(rnd2)

        
    aaaa = pd.concat(rnd_list_p)
    sns.stripplot(x="set", y="OE", data=aaaa, ax = axp, alpha=0.1)
    aaaa = pd.concat(aa_list_p)
    sns.stripplot(x="set", y="OE", data=aaaa, ax = axp, alpha=0.9,color="grey")

    
    aaaa = pd.concat(rnd_list_q)
    sns.stripplot(x="set", y="OE", data=aaaa, ax = axq, alpha=0.1)
    aaaa = pd.concat(aa_list_q)
    sns.stripplot(x="set", y="OE", data=aaaa, ax = axq, alpha=0.9,color="grey")

    #     axp.plot(x="set", y="OE",data=aaaa,markersize=15,facecolor="blue")#, ax = axp, alpha=0.999)
 
#     axp.bar(range(6),bar_signal(d1p[[slice(*p_range_zoom)]],cmpp)+bar_signal(d2p[[slice(*p_range_zoom)]],cmpp),color=list('rkbrkb'))
#     axq.bar(range(6),bar_signal(d1q[slice(*q_range_zoom_rel)],cmpq)+bar_signal(d2q[slice(*q_range_zoom_rel)],cmpq),color=list('rkbrkb'))

#     axprtb.bar(range(6),bar_signal(d1p[slice(*p_range_zoom)],rtpp)+bar_signal(d2p[slice(*p_range_zoom)],rtpp),color=list('rkbrkb'))
#     axqrtb.bar(range(6),bar_signal(d1q[slice(*q_range_zoom_rel)],rtpq)+bar_signal(d2q[slice(*q_range_zoom_rel)],rtpq),color=list('rkbrkb'))



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntRangeSlider(value=(0, 75), continuous_update=False, description='p_range_zoom', max=7…

In [51]:
def random_test(oe,comps,iters=100,thresh=0.1,cyclic=False):
    """
    a function that would generate a distribution of OE-averages
    for a number of randomized compartment tracks
    """
    oe1_list = []
    oe2_list = []
    oe3_list = []
    
    if cyclic:
        for i in range(len(comps)):
            # permute compartments with all the maskings etc
            comps_rnd = np.roll( comps, shift=i )
            oe1 = np.nanmean(oe[comps_rnd >= thresh])
            oe2 = np.nanmean(oe[(comps_rnd >= -thresh)&(comps_rnd < thresh)])
            oe3 = np.nanmean(oe[comps_rnd < -thresh])
            oe1_list.append(oe1)
            oe2_list.append(oe2)
            oe3_list.append(oe3)
    else:
        for i in range(iters):
            # permute compartments with all the maskings etc
            comps_rnd = np.random.permutation( comps )
            oe1 = np.nanmean(oe[comps_rnd >= thresh])
            oe2 = np.nanmean(oe[(comps_rnd >= -thresh)&(comps_rnd < thresh)])
            oe3 = np.nanmean(oe[comps_rnd < -thresh])
            oe1_list.append(oe1)
            oe2_list.append(oe2)
            oe3_list.append(oe3)
        
    oe1_df = pd.DataFrame({"OE":oe1_list})
    oe1_df["set"] = "A"
    oe2_df = pd.DataFrame({"OE":oe2_list})
    oe2_df["set"] = "AB"
    oe3_df = pd.DataFrame({"OE":oe3_list})
    oe3_df["set"] = "B"
    
    return pd.concat([oe1_df,oe2_df,oe3_df])
    
    


# # test (1)

# difflist_a1 = []
# difflist_a2 = []
# At = []
# Bt = []
# num_iters = 10000

# for iteration in range(num_iters):
#     aa = np.random.permutation( oe_eigen_df_filtered['compartment'].values )
#     oe_eigen_df_filtered['randomized_comp'] = pd.Series(aa)
#     grp_by_rnd_cmp = oe_eigen_df_filtered.groupby('randomized_comp')
#     gt = grp_by_rnd_cmp['allele1','allele2'].mean()
#     # first test expected:
#     all1_test = gt['allele1']
#     all2_test = gt['allele2']
#     all1_ABdiff = all1_test['A'] - all1_test['B']
#     all2_ABdiff = all2_test['B'] - all2_test['A']
#     difflist_a1.append(all1_ABdiff)
#     difflist_a2.append(all2_ABdiff)
#     # second test expected:
#     cmpA_test = gt.loc['A']
#     cmpB_test = gt.loc['B']
#     Atest = cmpA_test['allele1'] - cmpA_test['allele2']
#     Btest = cmpB_test['allele2'] - cmpB_test['allele1']
#     At.append(Atest)
#     Bt.append(Btest)

# # test (2)
# # this would be a different test probably ...
# # start = 1
# # maximum = num_iters # like 10000 or so

# # diffAlist = []
# # while start <= maximum:
# #     aa = oe_eigen_df_filtered['compartment'].reindex(np.random.permutation(oe_eigen_df_filtered['compartment'].index)).reset_index(drop=True)
# #     oe_eigen_df_filtered['randomized_comp'] = pd.Series(aa)
# #     randa1 = oe_eigen_df_filtered.loc[oe_eigen_df_filtered['randomized_comp'] == 'A', 'allele1'].mean()
# #     randa2 = oe_eigen_df_filtered.loc[oe_eigen_df_filtered['randomized_comp'] == 'A', 'allele2'].mean()
# #     diffA = randa1 - randa2
# #     #print (diffA)
# #     diffAlist.append(diffA)
# #     start += 1


In [28]:

# visualize compartments:
f1 = plt.figure(figsize=(9,4),constrained_layout=True)
sp1 = gridspec.GridSpec(ncols=1, nrows=2, height_ratios=[2,1], figure=f1)

axp1 = f1.add_subplot(sp1[0,0])

axpc1 = f1.add_subplot(sp1[1,0],sharex=axp1)


clr = samples["cool"][0]

chr1_parm = "chr1:0-74626886"

# chr1_qarm = "chr1:74626887-196202544"

p_range_zoom=widgets.IntRangeSlider(
        value=clr.extent(chr1_parm),
        min=0,
        max=clr.extent(chr1_parm)[1],
        step=1,
        continuous_update=False)


@widgets.interact(#chrom=clr.chromnames,
        p_range_zoom = p_range_zoom,
        log=True,
        balanced=True,
        samplez = samples["sample"].values,
)
def update(p_range_zoom,log,balanced,samplez):
    clr, = samples["cool"][samples["sample"] == samplez].values

    axp1.clear()
    axpc1.clear()
    
    # extract balacing weights for
    # alleles 1 and 2 - they are meaningless I guess
    # they only reflect - the total # of interactions
    # between allele1/2 and chr1
    
    if balanced:
        w1, = clr.bins().fetch("allele1")["weight"]
        w2, = clr.bins().fetch("allele2")["weight"]
        # print(w1,w2)
            
    # p-arm data:
    # obs:
    d1p, = clr.matrix(balance=balanced).fetch("allele1",chr1_parm)
    d2p, = clr.matrix(balance=balanced).fetch("allele2",chr1_parm)
    if balanced:
        d1p /= w1
        d2p /= w2
    if log:
        d1p = np.log(d1p)
        d2p = np.log(d2p)+0.2
    xp = range(*p_range_zoom)
    # compartments:
    cmpp = cmp_vals[slice(*p_range_zoom)]
    
    axp1.plot(xp, d1p[slice(*p_range_zoom)], 'gx', label="allele1", alpha=0.9)
    axp1.plot(xp, d2p[slice(*p_range_zoom)], 'mx', label="allele2", alpha=0.9)
    axp1.set_xlim(*p_range_zoom)
    axp1.legend(loc="best")
    
    fillcolor_compartment_style(cmp_vals, axpc1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntRangeSlider(value=(0, 150), continuous_update=False, description='p_range_zoom', max=…

In [52]:

# visualize compartments:
fc11 = plt.figure(figsize=(9,4),constrained_layout=True)
sp111 = gridspec.GridSpec(ncols=2, nrows=2,width_ratios=[4,1], height_ratios=[2,1], figure=fc11)


axp1 = fc11.add_subplot(sp111[0,0])

axpc1 = fc11.add_subplot(sp111[1,0],sharex=axp1)


axscat = fc11.add_subplot(sp111[0,1],sharey=axp1)
axbar = fc11.add_subplot(sp111[1,1])


clr = samples["cool"][0]

chr1_parm = "chr1:0-74626886"
# chr1_qarm = "chr1:74626887-196202544"

p_range_zoom=widgets.IntRangeSlider(
        value=clr.extent(chr1_parm),
        min=0,
        max=clr.extent(chr1_parm)[1],
        step=1,
        continuous_update=False)


sub_frac = widgets.FloatSlider(
        value=0.65,
        min=0,
        max=1.0,
        step=0.05)


@widgets.interact(#chrom=clr.chromnames,
        p_range_zoom = p_range_zoom,
        log=True,
        balanced=True,
        sub_frac = sub_frac,
        samplez = samples["sample"].values,
)
def update(p_range_zoom,log,balanced,sub_frac,samplez):
    clr, = samples["cool"][samples["sample"] == samplez].values

    axp1.clear()
    axpc1.clear()
    axscat.clear()
    axbar.clear()
    
    # extract balacing weights for
    # alleles 1 and 2 - they are meaningless I guess
    # they only reflect - the total # of interactions
    # between allele1/2 and chr1
    
    if balanced:
        w1, = clr.bins().fetch("allele1")["weight"]
        w2, = clr.bins().fetch("allele2")["weight"]
        # print(w1,w2)
            
    # p-arm data:
    # obs:
    d1p, = clr.matrix(balance=False).fetch("allele1",chr1_parm)
    d1p = np.random.binomial(d1p,sub_frac).T
    if balanced:
        d1p = d1p * clr.bins().fetch(chr1_parm)[weight_name].values * w1

    d2p, = clr.matrix(balance=balanced).fetch("allele2",chr1_parm)
    # exp:
    ep1, = pexp[:,ins_row].T
    ep2, = pexp[:,ins_row].T
    #     ep1, = pexp[:,725].T
    #     ep2, = pexp[:,725].T
    
    if balanced:
        #         d1p /= w1
        #         d2p /= w2
        #         ep1 /= w1
        #         ep2 /= w2
        # Obs/Exp :
        d1p = np.divide(d1p,ep1)
        d2p = np.divide(d2p,ep2)
        oe1p = d1p.copy()
        oe2p = d2p.copy()
    if log:
        d1p = np.log(d1p)
        d2p = np.log(d2p)#+0.2

    a1_vis = d1p[slice(*p_range_zoom)]
    a2_vis = d2p[slice(*p_range_zoom)]
    oe1_vis = oe1p[slice(*p_range_zoom)]
    oe2_vis = oe2p[slice(*p_range_zoom)]
    xp = range(*p_range_zoom)
    
    axp1.bar(xp, a1_vis, label="allele1", alpha=0.6)#, 'gx-', label="allele1", alpha=0.9)
    axp1.bar(xp, a2_vis, label="allele2", alpha=0.3)#, 'mx-', label="allele2", alpha=0.9)
    axp1.set_xlim(*p_range_zoom)
    axp1.legend(loc="best")
    
    axscat.scatter(a1_vis,a2_vis,alpha=0.4)
    
    # compartments:
    cmpp = cmp_vals[slice(*p_range_zoom)]
    fillcolor_compartment_style(cmp_vals, axpc1, bin_range=p_range_zoom)
    axpc1.set_xlim(*p_range_zoom)
    
    
    #     [a1A,a2A,a1B,a2B]
    # ...
    a1A = np.nanmean(oe1_vis[cmpp >= 0.1])
    a1AB = np.nanmean(oe1_vis[(cmpp >= -0.1)&(cmpp < 0.1)])
    a1B = np.nanmean(oe1_vis[cmpp < -0.1])

    a2A = np.nanmean(oe2_vis[cmpp >= 0.1])
    a2AB = np.nanmean(oe2_vis[(cmpp >= -0.1)&(cmpp < 0.1)])
    a2B = np.nanmean(oe2_vis[cmpp < -0.1])

    axbar.bar(range(6),[a1A,a1AB,a1B,a2A,a2AB,a2B],color=list('rrrggg'))
#     print([a1A,a2A,a1B,a2B])
#     matplotlib.pyplot.bar(x, height, width=0.8, bottom=None, *, align='center', data=None, **kwargs)[source]

/home/venevs/miniconda3/envs/newexp/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntRangeSlider(value=(0, 75), continuous_update=False, description='p_range_zoom', max=7…

In [ ]:
frg = plt.figure(figsize=(3,3))
plt.hist(cmp_vals,bins=50)

In [ ]:
plt.bar?

In [ ]:
def bar_AB(ev,ar1,ar2):
    

## why A-B bar plots might be misleading:

1. definition of compartments might be impresize - i.e. a dip in A, might actually be a small B-compartment
2. when comparing Obs/Exp for allele1 and allele2 - we might be off because allele2 is shifted up overall - why is that is a separate question ...

so it seems like the best thing we could do now is to carefully look at the Obs for both and see if they are "in-phase" or in "opposite phases" with each other ...

... and if that observation is consistent across samples ...

### we'll do them anyways !!!


In [ ]:
#add compartment status info
conditions = [
    (oe_eigen_df['E1'] >= -0),
    (oe_eigen_df['E1'] < -0)]

choices = ['A', 'B']
oe_eigen_df['compartment'] = np.select(conditions, choices)

In [ ]:
np.random.binomial([1,2,3,4,5,6,7,8,9],0.5)

# DO NOT LOOK BELOW - INCOMPREHENSIBLE DOWN THERE ...

there is a little mess down below ...

In [ ]:
# RE-READ SOME COOLERS AT LOWER RESOLUTION 1MB TO DO SOME QUICK CALCULATIONS ...
binsize = 1_000_000
res_human = f"{int(binsize/1000)}kb"

cool_path = "/home/venevs/DOTS_TESTING/alv-chicken/oldref-20200410"
get_cpath = lambda c: os.path.join(cool_path,c+f"::/resolutions/{binsize}")

samples["cool"] = [cooler.Cooler(get_cpath(c)) for c in samples["mcool"]]


samples["sum"] = [s.info["sum"] for s in samples["cool"]]

In [ ]:
# trans interactions (whole chromosome ...)
f,ax = plt.subplots()


for i in range(2,10):
    c = samples["cool"][i]
    balance = False
    cname = "balanced" if balance else "count"
    hh = c.matrix(balance=balance,as_pixels=True,sparse=True)
    x = []
    y = []
    for _c0,_c2 in itertools.combinations(c.chromnames[:-3],2):
        _x = float(c.chromsizes[_c0])*c.chromsizes[_c2]
        _y = hh.fetch(_c0,_c2)[cname].sum()#/c.info["sum"]
        x.append(_x)
        y.append(_y)
    ax.plot(x,y,"o-",alpha=0.7,label=f"{samples['sample'][i]}")


ax.legend(loc="best")
ax.set_xlabel("inter chromosomal area, bp^2")
ax.set_ylabel("# of interactions")
ax.set_title("trans interactions, chromosome level")

In [ ]:
# cis interactions - whole chromosome ...
# trans interactions (whole chromosome ...)
f,ax = plt.subplots(figsize=(9,5))


for i in range(3,10):
    c = samples["cool"][i]
    balance = True
    cname = "balanced" if balance else "count"
    hh = c.matrix(balance=balance,as_pixels=True,sparse=True)
    x = []
    y = []
    for chrom in c.chromnames[:-3]:
        _x = float(c.chromsizes[chrom])
        _y = np.nansum(hh.fetch(chrom)[cname])#/c.info["sum"]
        x.append(_x)
        y.append(_y)
    ax.plot(x,y,"o-",label=f"{samples['sample'][i]}")
    
    
ax.legend(loc="best",frameon=False)
ax.set_xlabel("chrom size, bp")
ax.set_ylabel("# of interactions")
ax.set_title("cis interactions, chromosome level")

In [ ]:
samples

In [ ]:
cis_sum = []
for c in samples["cool"]:
    b = c.bins()[:]
    p = c.pixels()[:]
    cis_mask = b["chrom"][p["bin1_id"].values].values == b["chrom"][p["bin2_id"].values].values
    s = p[cis_mask]["count"].sum()
    cis_sum.append(s)

samples["cis"] = cis_sum

## samples stats ... overall

In [ ]:
def get_a12_stats(clr):
    sp_a1 = clr.matrix(balance=False).fetch("allele1").sum()
    sp_a2 = clr.matrix(balance=False).fetch("allele2").sum()
    c1p_a1 = clr.matrix(balance=False).fetch("allele1","chr1").sum()
    c1p_a2 = clr.matrix(balance=False).fetch("allele2","chr1").sum()
    tp_a1 = clr.matrix(balance=False)[clr.offset("allele1"),:].sum()
    tp_a2 = clr.matrix(balance=False)[clr.offset("allele2"),:].sum()
    mmm = clr.matrix(balance=False)[clr.offset("chrM"),:].sum()
    self_mmm = clr.matrix(balance=False).fetch("chrM","chrM").sum()
    #
    ret_dict = {
        "self1":sp_a1,
        "self2":sp_a2,
        "chr1-a1":c1p_a1,
        "chr1-a2":c1p_a2,
        "trans-a1":tp_a1 - sp_a1 - c1p_a1,
        "trans-a2":tp_a2 - sp_a2 - c1p_a2,
        "chrM":mmm-self_mmm,
    }
    return pd.Series(ret_dict)



samples_allele = pd.merge(
                    samples,
                    samples[~samples["sample"].str.contains("WT")]["cool"].apply(get_a12_stats),
                    how="outer",
                    left_index=True,
                    right_index=True,
                )

In [ ]:
samples_allele

In [ ]:
fsc = plt.figure(figsize=(7,7))
axsc = fsc.add_subplot()

cols = ["self1",
"self2",
"chr1-a1",
"chr1-a2",
"trans-a1",
"trans-a2"]

pd.plotting.scatter_matrix(samples_allele[cols].iloc[2:],ax=axsc,marker="s")#,marker_size=8)

In [ ]:
f,ax = plt.subplots(figsize=(9,5),constrained_layout=True)

cols = ["self1",
"self2",
"chr1-a1",
"chr1-a2",
"trans-a1",
"trans-a2",
"chrM"]

txt_feature = widgets.Text(
    value=','.join(["sum","cis"]+cols),
    description='Features:',
    disabled=False
)


@widgets.interact(log=True,features=txt_feature)
def update(log,features):
    ax.clear()
    feats_to_show = features.split(',')
    samples_allele.plot(x="sample",y=feats_to_show,kind="bar",ax=ax,logy=log)
    
    
print("choose features:"+','.join(["sum","cis"]+cols))

In [ ]:
# !curl https://hgdownload.soe.ucsc.edu/goldenPath/galGal5/database/cytoBandIdeo.txt.gz | gunzip

# mat = samples["cool"][1].matrix().fetch("chr1:0-74626886","chr1:0-74626886")
# plt.imshow(np.log(mat),cmap="YlOrBr",vmin=-12,vmax=-1)

chr1_parm = "chr1:0-74626886"

mat = samples["cool"][0].matrix().fetch(chr1_parm,chr1_parm)
plt.imshow(np.log(mat),cmap="YlOrBr")

In [ ]:
mat = samples["cool"][0].matrix().fetch("chr1:9000000-10000000",chr1_parm)
plt.imshow(np.log(mat),cmap="YlOrBr")

In [ ]:
# import higlass
# import higlass.tilesets
# from higlass.client import Track, View

In [ ]:
host="vangogh.ad.umassmed.edu"
some_port = "12347"
allip = "0.0.0.0"
server = f"http://{host}:{some_port}/api/v1/"
hw,hh = 6,300

nrows = 2
ncols = 2

tss = []
trs = []
views = []

for idx,i in enumerate([6,7,8,9]):
    ts = higlass.tilesets.cooler(get_cpath(samples["mcool"][i]).split("::")[0])
    tr = Track('heatmap', position='center',width=hw,height=hh, tileset=ts, server=server,options={"name":samples["sample"][i]})
    row = idx%ncols
    col = idx%nrows
    x = row*hw
    y = col*hh
    view = View([tr,],x=x,y=y,width=hw, initialXDomain=[0,70_000_000],initialYDomain=[0,70_000_000])
    views.append(view)

display, server, viewconf = higlass.display(views,location_syncs=[views,],zoom_syncs=[views,], host=allip, server_port=some_port)
display